In [3]:
import database as db
from table import print_table

In [3]:
res = await db.execute("""
    create table if not exists locations (
        location_id int PRIMARY KEY,
        city varchar(30),
        postal_code varchar(12)
    );
""")

In [ ]:
res = await db.execute("""
    insert into locations
    values
    (1, 'Roma', '00989'),
    (2, 'Venice', '10934'),
    (3, 'Tokyo', '1689'),
    (4, 'Hiroshima', '6823'),
    (5, 'Southlake', '26192'),
    (6, 'South San Francisco', '99236'),
    (7, 'South Brunswick', '50090'),
    (8, 'Seattle', '98199'),
    (9, 'Toronto', 'M5V 2L7'),
    (10, 'Whitehorse', 'YSW 9T2');
""")

print(f"Added {res.rowcount} rows")

Added 10 rows


In [18]:
res = await db.execute("select * from locations;")
print_table(["id", "city", "postal_code"], res.fetchall())


+----+---------------------+-------------+
| id |         city        | postal_code |
+----+---------------------+-------------+
| 1  |         Roma        |    00989    |
| 2  |        Venice       |    10934    |
| 3  |        Tokyo        |     1689    |
| 4  |      Hiroshima      |     6823    |
| 5  |      Southlake      |    26192    |
| 6  | South San Francisco |    99236    |
| 7  |   South Brunswick   |    50090    |
| 8  |       Seattle       |    98199    |
| 9  |       Toronto       |   M5V 2L7   |
| 10 |      Whitehorse     |   YSW 9T2   |
+----+---------------------+-------------+


In [ ]:
res = await db.execute("""
    alter table employees
    add location_id int,
    add foreign key(location_id) references locations(location_id);
""")

In [ ]:
await db.execute("""
    create sequence seq
    increment 1
    minvalue 1
    maxvalue 10
    start with 1
    cycle;
""")

res = await db.execute("""
    update employees
    set location_id = nextval('seq')
    returning *;
""")

print_table(["id", "first_name", "last_name", "job_id", "salary", "manager_id", "department_id", "location_id"], res.fetchall())

+----+-------------+-------------+------------+--------+------------+---------------+-------------+
| id |  first_name |  last_name  |   job_id   | salary | manager_id | department_id | location_id |
+----+-------------+-------------+------------+--------+------------+---------------+-------------+
| 1  |    Steven   |     King    |  AD_PRES   | 24000  |    None    |       90      |      1      |
| 2  |    Neena    |   Kochhar   |   AD_VP    | 17000  |     1      |       90      |      2      |
| 3  |     Lex     |   De Haan   |   AD_VP    | 17000  |     1      |       90      |      3      |
| 4  |  Alexander  |    Hunold   |  IT_PROG   |  9000  |     3      |       60      |      4      |
| 5  |    Bruce    |    Ernst    |  IT_PROG   |  6000  |     4      |       60      |      5      |
| 6  |    David    |    Austin   |  IT_PROG   |  4800  |     4      |       60      |      6      |
| 7  |    Valli    |  Pataballa  |  IT_PROG   |  4800  |     4      |       60      |      7      |


In [27]:
res = await db.execute("""
    select employee_id, first_name || ' ' || last_name as full_name
    from employees
    where last_name like '% %';
""")

print_table(["id", "full_name"], res.fetchall())

+----+-------------+
| id |  full_name  |
+----+-------------+
| 3  | Lex De Haan |
+----+-------------+


In [28]:
res = await db.execute("""
    select department_name, min(salary) as min_salary
    from employees
    join departments using(department_id)
    group by department_name
    having min(salary) > 10000
    order by min_salary desc;
""")

print_table(["department_name", "min_salary"], res.fetchall())

+-----------------+------------+
| department_name | min_salary |
+-----------------+------------+
|    Executive    |   17000    |
|    Marketing    |   13000    |
|      Sales      |   10500    |
+-----------------+------------+


In [32]:
res = await db.execute("""
    select city, count(employee_id) as employees_count, round(avg(salary), 2) as salary_avg
    from employees
    join locations using(location_id)
    group by city
    order by salary_avg desc;
""")

print_table(["city", "employees_count", "salary_avg"], res.fetchall())

+---------------------+-----------------+------------+
|         city        | employees_count | salary_avg |
+---------------------+-----------------+------------+
|        Venice       |        6        |  8650.00   |
|         Roma        |        6        |  8316.67   |
|        Tokyo        |        6        |  7400.00   |
|       Toronto       |        5        |  6201.60   |
|      Hiroshima      |        5        |  5660.00   |
|      Southlake      |        5        |  5640.00   |
|      Whitehorse     |        5        |  5600.00   |
| South San Francisco |        5        |  5500.00   |
|   South Brunswick   |        5        |  5220.00   |
|       Seattle       |        5        |  5000.00   |
+---------------------+-----------------+------------+


In [5]:
res = await db.execute("""
    select * from select_data(30);
""")

print_table(["department_id", "department_name", "manager_id"], res.fetchall())

+---------------+-----------------+------------+
| department_id | department_name | manager_id |
+---------------+-----------------+------------+
|       40      | Human Resources |     53     |
|       50      |     Shipping    |     22     |
|       60      |        IT       |     4      |
|       80      |      Sales      |     46     |
|       90      |    Executive    |     1      |
|      100      |     Finance     |     9      |
+---------------+-----------------+------------+


In [7]:
await db.execute("""
    create or replace function select_data_1(min_id int) returns setof departments as $$
        select *
        from departments
        where department_id > min_id;
    $$ language sql;
""")

res = await db.execute("""
    select *
    from select_data_1(30);
""")
print_table(["department_id", "department_name", "manager_id"], res.fetchall())

+---------------+-----------------+------------+
| department_id | department_name | manager_id |
+---------------+-----------------+------------+
|       40      | Human Resources |     53     |
|       50      |     Shipping    |     22     |
|       60      |        IT       |     4      |
|       80      |      Sales      |     46     |
|       90      |    Executive    |     1      |
|      100      |     Finance     |     9      |
+---------------+-----------------+------------+


In [9]:
res = await db.execute("""
    select * from get_job_stats_pg();
""")

print_table(["job_title", "employees_count"], res.fetchall())

+--------------------------------+-----------------+
|           job_title            | employees_count |
+--------------------------------+-----------------+
|          Stock Clerk           |        20       |
|        Purchasing Clerk        |        5        |
|         Sales Manager          |        5        |
|         Stock Manager          |        5        |
|           Accountant           |        5        |
|           Programmer           |        5        |
| Administration Vice President  |        2        |
|        Finance Manager         |        1        |
|           President            |        1        |
| Human Resources Representative |        1        |
|    Administration Assistant    |        1        |
|       Marketing Manager        |        1        |
|       Purchasing Manager       |        1        |
+--------------------------------+-----------------+


In [10]:
await db.execute("""
    create or replace function get_job_stats_py()
    returns TABLE (job_title varchar(35), employees_count int)
    as $$
        select job_title, count(job_id) as employees_count
        from employees
        join jobs using(job_id)
        group by job_title
        order by employees_count desc;
    $$ language sql;
""")

res = await db.execute("""
    select *
    from get_job_stats_py();
""")

print_table(["job_title", "employees_count"], res.fetchall())

+--------------------------------+-----------------+
|           job_title            | employees_count |
+--------------------------------+-----------------+
|          Stock Clerk           |        20       |
|        Purchasing Clerk        |        5        |
|         Sales Manager          |        5        |
|         Stock Manager          |        5        |
|           Accountant           |        5        |
|           Programmer           |        5        |
| Administration Vice President  |        2        |
|        Finance Manager         |        1        |
|           President            |        1        |
| Human Resources Representative |        1        |
|    Administration Assistant    |        1        |
|       Marketing Manager        |        1        |
|       Purchasing Manager       |        1        |
+--------------------------------+-----------------+
